# Generate new images using fine-tuned model

In [1]:
from diffusers import StableDiffusionPipeline
import torch
#from google.colab import drive
import os

/Users/victormicha/PythonProjects/IP_Paris/CVGenerativeAI_EP/GenAIBrainMRI/pytorch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.backends.mps.is_available():
    device = torch.device("mps")
print(f"Using device: {device}")

Using device: mps


## TO GET MODEL FROM GOOGLE DRIVE

In [3]:
# GET FINE-TUNED MODEL FROM GOOGLE DRIVE
models = [
    ('fine_tuned_model_A100_800', torch.float16),  # A100 TRAINED ON FP16
    ('fine_tuned_model_MPS_248', torch.float32) # MPS trained on FP32
]

model_type, dtype= models[0]

drive.mount('/content/drive')


model_path = os.path.join('/content/drive/MyDrive/', model_type)

# Check if the folder exists
if os.path.exists(model_path):
  print(f"The folder '{model_path}' exists.")
else:
  print(f"The folder '{model_path}' does not exist. Please check the path.")


Mounted at /content/drive
The folder '/content/drive/MyDrive/fine_tuned_model_A100_800' exists.


'if os.path.exists(checkpoint_path):\n    print(f"The folder \'{checkpoint_path}\' exists.")\nelse:\n    print(f"The folder \'{checkpoint_path}\' does not exist. Please check the path.")\n    '

In [4]:
print(model_path)
GPU_numImagesTrainedOn = model_type[17:] # GPU_numImagesTrainedOn
print(GPU_numImagesTrainedOn)

/content/drive/MyDrive/fine_tuned_model_A100_800
A100_800


In [5]:
!ls {model_path}

checkpoint-2000  feature_extractor  model_index.json  scheduler     tokenizer  vae
checkpoint-4000  logs		    safety_checker    text_encoder  unet


In [6]:
!ls {model_path}/unet

config.json  diffusion_pytorch_model.safetensors


## TO GET FINE TUNED MODEL FROM CURRENT DIR

In [5]:
model_path='fine_tuned_model_MPS_248'
GPU_numImagesTrainedOn = 'MPS_248'
dtype=torch.float32

## Generate new images!

In [6]:
# Load fine-tuned model

sd_model_ft = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=dtype)
sd_model_ft = sd_model_ft.to(device)

# Define prompts (same as your test)
prompts = [
    "MRI of brain with glioma tumor",
    "MRI of brain with no tumor"
]

# Generate post-fine-tuning test images
dir_name = f"post_fine_tuning_test_images_{GPU_numImagesTrainedOn}"
os.makedirs(dir_name, exist_ok=True)
for i, prompt in enumerate(prompts):
    image = sd_model_ft(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
    image.save(f"{dir_name}/test_image_{i}.jpg")
    print(f"Generated: test_image_{i}.jpg with prompt: '{prompt}'")


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:44<00:00,  1.11it/s]


Generated: test_image_0.jpg with prompt: 'MRI of brain with glioma tumor'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:46<00:00,  1.08it/s]


Generated: test_image_1.jpg with prompt: 'MRI of brain with no tumor'
